In [ ]:
import pandas as pd
import DB as db
import numpy as np

# test 1

In [ ]:
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2022-12-11.zip"
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-20.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-19.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-18.zip"

logdf= pd.read_csv(log_summary_file_name, low_memory=False).query("token == 'confirmLandReservation True'")\
    .drop(columns=['node', 'task_id','project_id', 'error_line'])
# logdf.head()

In [ ]:
# logdf.query("Land_ID in (80312, 80487)")
logdf.query("NID == '26312132702012'")

In [ ]:
land_app_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\DB-summary\DB checksum\NewQueryLand.zip"
land_app_fn =r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\checksum\NewQueryLand.zip"
dbdf= pd.read_csv(land_app_fn, low_memory=False, dtype={'applicant_national_id':str}).query("project_id == 86")
    # .query("project_id == 86 and land_application_status == 'W'")[['applicant_national_id', 'check_sum', 'land_id', 'project_id']]

In [ ]:
dbdf.query("land_id == '80487'")

In [ ]:
df = (dbdf.merge(logdf, left_on='applicant_national_id', right_on='NID')
      .assign(match = lambda x:(x.checksum == x.check_sum))
    #   .fillna('land_id', axis=1)
    #   .assign(land_id = lambda x: x.astype(np.int64) )
    )
# df.info()

In [ ]:
df.match.value_counts()

In [ ]:
df.query("match == False").assign(land_id = lambda x: x.land_id.astype(np.int64)).to_excel(r"C:\Users\yahia\Downloads\mismatch.xlsx")

## UNITS

In [ ]:
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-02-07.zip"

logdf= pd.read_csv(log_summary_file_name, low_memory=False, dtype={'NID':str}).query("token == 'confirmReservation True'")\
    .drop(columns=['node', 'task_id','project_id', 'error_line'])



In [ ]:
logdf.query("NID == '26411200102736'")

In [ ]:
logdf.token.value_counts()
logdf.to_csv('./out/mis_log.csv')

In [ ]:
unit_app_fn = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\checksum\checksum.zip"
dbdf= pd.read_csv(unit_app_fn, low_memory=False, dtype={'applicant_national_id':str}).query("project_id == 87")

In [ ]:
dbdf.query("applicant_national_id == '26411200102736'")

In [ ]:
dbdf.project_id.value_counts().sort_index(ascending=False)

In [ ]:
dbdf.any()

In [ ]:
df = (dbdf.merge(logdf, how='outer', left_on='applicant_national_id', right_on='NID')
      # .assign(match = lambda x:(x.checksum == x.check_sum))
)


In [ ]:
df.to_csv('./out/checksum.csv')

In [ ]:
df.match.value_counts()

In [ ]:
df.query("match == False").assign(unit_id = lambda x: x.unit_id.astype(np.int64))[['NID', 'check_sum', 'checksum']]
#.to_excel(r"C:\Users\yahia\Downloads\mismatch.xlsx")

# LE04

In [ ]:
le04 = pd.read_excel(r"C:\Users\yahia\Downloads\le04.xls", skiprows=1)

In [ ]:
le04.iloc[:,[0,1,2,3,17]].rename({
    'نسبة التميز': 'excellence_ratio',
    'مساحة الأرض':'land_size',
    'رقم الأرض':'Land_No',
    'كود المجاورة':'Sub_District',
    'الرقم القومى':'NID'
}, axis=1)


### new zip file

In [ ]:
projdf = (db.query_to_pd("project")
        .set_index('project_id')
        .drop(columns=['index','Unnamed: 3','Unnamed: 11','Unnamed: 13'], axis=1)
        .assign (start_date = lambda x: x.start_date.str[:10])
        .assign (end_date= lambda x: x.end_date.str[:10])
        .assign (publish_date= lambda x: x.publish_date.str[:10])
        .rename(columns={'project_type_name_ar':'proj_type', 'No. of reservations':'n_Resv'})
        .assign(n_Resv = lambda x: x.fillna(0).n_Resv.astype(int))
        .assign(_ = lambda x: x.insert(0,'select', False) )
        .sort_values(by='start_date', ascending=False)
    )
# projdf.insert(0,'select', False)
# projdf['# Res.'].fillna(0).astype(int)
projdf.head(4)

# test 1

In [ ]:
rep_date = "07-08-2023  10:29:27 AM"
rep_date_df = pd.DataFrame(data=[rep_date], columns=['rep_date']).set_index('rep_date')
print ("rep_date_df ****:\n", rep_date_df)
db.df_to_sql(rep_date_df, table = 'project_rep_date', if_exists='append')

In [ ]:
rep_date_df